<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#封装成函数" data-toc-modified-id="封装成函数-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>封装成函数</a></span><ul class="toc-item"><li><span><a href="#read-news" data-toc-modified-id="read-news-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>read news</a></span></li><li><span><a href="#read-similars-of-&quot;say&quot;" data-toc-modified-id="read-similars-of-&quot;say&quot;-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>read similars of "say"</a></span></li><li><span><a href="#split-one-piece-of-news" data-toc-modified-id="split-one-piece-of-news-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>split one piece of news</a></span><ul class="toc-item"><li><span><a href="#load-LTP" data-toc-modified-id="load-LTP-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>load LTP</a></span></li><li><span><a href="#split-each-news" data-toc-modified-id="split-each-news-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>split each news</a></span></li></ul></li></ul></li></ul></div>

# 封装成函数

[Reference1 - other solutions](https://github.com/enix223/nlp-course/blob/master/notebooks/news_options.ipynb)

[Reference2 - pyltp](https://pyltp.readthedocs.io/zh_CN/latest/api.html#id13)

## read news

In [ ]:
import pandas as pd
NEWS_DATA_PATH = 'mydata/sqlResult_1558435.csv'
df = pd.read_csv(NEWS_DATA_PATH, encoding='gb18030')
contents = df[~df['content'].isnull()]['content']
#use contents (list) as input of the function

## read similars of "say"

In [ ]:
import pandas as pd
say_similar = pd.read_csv("similar_said.csv", encoding = "gbk")
said_list = list(say_similar['similar_of_said'])

## split one piece of news

### load LTP

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import jieba
import pyltp
import pandas as pd
from gensim.models.word2vec import Word2Vec

LTP_DATA_DIR = 'mydata/ltp_data_v3.4.0'  # ltp模型目录的路径
srl_model_path = os.path.join(LTP_DATA_DIR, 'pisrl.model')

segmentor = pyltp.Segmentor()
segmentor.load(cws_model_path)

postagger = pyltp.Postagger()
postagger.load(pos_model_path)

recognizer = pyltp.NamedEntityRecognizer()
recognizer.load(ner_model_path)

parser = pyltp.Parser()
parser.load(par_model_path)

labeller = pyltp.SementicRoleLabeller() # 初始化实例
labeller.load(srl_model_path)  # 加载模型

print("########################end load#################################")


### split each news

In [ ]:
news = contents[96]
words = segmentor.segment(news.replace('\r\n', ''))
postags = postagger.postag(words)
netags = recognizer.recognize(words, postags)

#find "say"
say_positions = [(w, i) for i, w in enumerate(words) if w in said_list]

#find entities who say that
all_names_positions = [(i, tag) for i, tag in enumerate(netags) if 'Nh' in tag]
all_names = [(words[name[0]], name[0]) for name in all_names_positions]
names = set(map(lambda x: x[0], all_names))

#find each sentence(background+saying+entity_say) in one group of words(split by \r\n)
arcs = parser.parse(words, postags)

#extract from name
def get_all_words_related_to(index):
    return [words[i] for i, arc in enumerate(arcs) if arc.head == index]

candidates = [(name, arcs[name[1]].relation, arcs[name[1]].head) 
              for name in all_names if arcs[name[1]].relation == 'SBV']

def is_sentence_end(w):
    return w in ["。", "!", "！"]

#whole extract
options = []
for say_word, pos in say_positions:
    option = {'say': say_word}
    # Get the name who say the words
    for i in range(pos, 0, -1):
        w = words[i]
        if w in names:
            option['name'] = w
            break
    
    sentence = ''
    for w in words[pos + 1:]:
        sentence += w
        if is_sentence_end(w):
            option['sentence'] = sentence
            break
    
    options.append(option)
    
print(option)#option(dict) include "name", "say" and "sentence"    